In [1]:
from pysus.ftp.databases.sinan import SINAN
from pathlib import Path
pysus_path = Path().resolve().parents[0] / "data" / "raw"

sinan = SINAN().load()  # Loads the files from DATASUS

In [2]:
sinan.metadata

{'long_name': 'Doenças e Agravos de Notificação',
 'source': 'https://portalsinan.saude.gov.br/',
 'description': 'The Notifiable Diseases Information System - Sinan is primarilyfed by the notification and investigation of cases of diseases and conditions listed in the national list of compulsorily notifiable diseases (Consolidation Ordinance No. 4, September 28, 2017, Annex).However, states and municipalities are allowed to include other important health problems in their region, such as difilobotriasis in the municipality of São Paulo. Its effective use enables the dynamic diagnosis of the occurrence of an event in the population, providing evidence for causal explanations of compulsorily notifiable diseases and indicating risks to which people are exposed. This contributes to identifying the epidemiological reality of a specific geographical area. Its systematic, decentralized use contributes to the democratization of information, allowing all healthcare professionals to access and 

In [3]:
sinan.get_files(dis_code=["ZIKA", "CHIK"], year=[2022, 2023])

[CHIKBR22.dbc, CHIKBR23.dbc, ZIKABR22.dbc, ZIKABR23.dbc]

In [5]:
files = sinan.get_files(dis_code=["ZIKA", "CHIK"], year=[2022, 2023])
sinan.download(files, local_dir= str(pysus_path))

  0%|          | 0/4 [00:00<?, ?it/s]

ZIKABR23.parquet: 100%|██████████| 17.7k/17.7k [00:00<00:00, 37.6kB/s]


[/Users/guttofranca/Repos/dq-sus/data/raw/CHIKBR22.parquet,
 /Users/guttofranca/Repos/dq-sus/data/raw/CHIKBR23.parquet,
 /Users/guttofranca/Repos/dq-sus/data/raw/ZIKABR22.parquet,
 /Users/guttofranca/Repos/dq-sus/data/raw/ZIKABR23.parquet]